In [1]:
import math
import numpy as np
import scipy as sp
from scipy.integrate import odeint

Assumptions:

1. Perfect thermal contact between the boundaries (i.e, the temperature at the mating surfaces are equal)
2. The temperature gradient in the polypropylene glass is assumed to be constant due to negligible thickness, i.e lumped capacitance analysis can be used in this case
3. 

In [12]:
# Given parameters (_t thickness m) (_k conductivity W/mK) (_c sp. heat capacity KJ/kg) (_lh latent heat KJ/kg) (_rho density kg/m^3)

# Outer Insulation
minWool_t = 50e-3
minWool_k = 34e-3

# PolyPropylene 
Glass_t = 3.5e-3
Glass_k = 34e-3

# Dry Ice
co2_t = 60e-3
co2_k = 16e-3
co2_c = 0.658
co2_rho = 1.795

# PCM
PCM_c_liq = 3.40
PCM_c_sol = 1.87
PCM_k_liq = 0.53
PCM_k_sol = 5.26
PCM_lh = 327
PCM_rho = 1043 

In [13]:
def volume(r1,r2):
    return abs((4/3)*np.pi*(pow(r2/2,3) - pow(r1/2,3)))

def surfArea(r):
    return 4*np.pi*pow(r,2)